In [1]:
import os


In [2]:
%pwd

'c:\\ML_OPS\\Projects\\project1\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\ML_OPS\\Projects\\project1'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [17]:
from src.DataScienceProject.constants import *

from src.DataScienceProject.utils.shared import read_yaml, create_directories


In [25]:
%load_ext autoreload
%autoreload 2


In [27]:
from src.DataScienceProject.utils.shared import read_yaml, create_directories


In [28]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories(self.config.artifacts_root)

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source.url,
            local_data_file= config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config
    
 


In [29]:
import os
import urllib.request as request
from src.DataScienceProject import logger
import zipfile

In [31]:
# component-data Ingestion
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    
    # Dwnloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
          filename,headers = request.urlretrieve(
              url= self.config.source_url,
              filename=self.config.local_data_file
          )
          logger.info(f'{filename} download! with following info: \n{headers}')
        else:
            logger.info(f"File Already exists")  
    
    def extract_zip_file(self):
        """Zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f'Zip file extracted to {unzip_path}')



In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise  e
